In [2]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import AutoModel

model_ckpt = "bert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 6

model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device))

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}


In [4]:
%%capture
%pip install datasets

In [5]:
from datasets import load_dataset
emotions = load_dataset("emotion")

README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [6]:
import pandas as pd
emotions.set_format(type="pandas")
df = emotions["train"][:]
df.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [16]:
from transformers import BertTokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [28]:
emotions.reset_format()

In [29]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [32]:
# emotions_encoded['train'][:10]

In [33]:
from transformers import Trainer, TrainingArguments
batch_size = 64

logging_steps = len(emotions_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=True,
                                  log_level="error")


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [34]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=emotions_encoded["train"],
                  eval_dataset=emotions_encoded["validation"],
                  tokenizer=tokenizer)
trainer.train();

<ipython-input-34-3c0a11c6afca>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=training_args,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.796500,0.272834,0.912500,0.913031
2,0.208500,0.185854,0.929000,0.929080


In [51]:
preds_output = trainer.predict(emotions_encoded["validation"])

In [52]:
import numpy as np
preds_output

PredictionOutput(predictions=array([[ 5.1829104 , -0.28028765, -0.55377823, -1.0418843 , -1.1249135 ,
        -1.7996291 ],
       [ 5.1259828 , -0.15406136, -1.0995164 , -0.7820315 , -1.1827427 ,
        -1.4603488 ],
       [-1.2264132 ,  2.206082  ,  3.2166357 , -1.3187457 , -2.1041117 ,
        -1.6873349 ],
       ...,
       [-0.56390697,  5.276025  ,  0.4216017 , -1.4465593 , -1.594943  ,
        -0.3984712 ],
       [-2.059636  ,  3.06082   ,  2.6448667 , -1.543726  , -1.747968  ,
        -1.1629143 ],
       [-1.2125951 ,  5.2770376 ,  0.33406007, -1.5928546 , -1.3033245 ,
         0.2045808 ]], dtype=float32), label_ids=array([0, 0, 2, ..., 1, 1, 1]), metrics={'test_loss': 0.1858539879322052, 'test_accuracy': 0.929, 'test_f1': 0.9290796184593119, 'test_runtime': 7.1438, 'test_samples_per_second': 279.964, 'test_steps_per_second': 4.479})

In [70]:
preds_output.metrics

{'test_loss': 0.1858539879322052,
 'test_accuracy': 0.929,
 'test_f1': 0.9290796184593119,
 'test_runtime': 7.1438,
 'test_samples_per_second': 279.964,
 'test_steps_per_second': 4.479}

In [53]:
 # how use values for submit
 preds = np.argmax(preds_output.predictions, axis=1)
 preds

array([0, 0, 2, ..., 1, 1, 1])

## Predict Using Just One Sentence

In [62]:
Sentence = "I love HIM"
Stk = tokenizer(Sentence)
Stk

{'input_ids': [101, 1045, 2293, 2032, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [63]:
y = trainer.predict([Stk])

sadness (0), joy (1), love (2), anger (3), fear (4), and surprise (5).

In [64]:
np.argmax(y.predictions, axis=1)

array([2])